# LLM Session Memory - Scope

Managing user input requires attention to access patterns and controling which applications can access which pieces of data. To help with this, the `SessionManager` class includes the `scope` parameter that controls the level of shared access.
We'll use the example of a travel agent application to illustrate different access levels.

Start by creating a simple wrapper around your LLM provider of choice. In this case we'll use OpenAI.

In [13]:
from typing import Dict, List
import os
from openai import OpenAI

class OpenAIClient():
    def __init__(self, api_key: str = None, model:str ='gpt-3.5-turbo'):
        api_key = api_key or os.getenv("OPENAI_API_KEY")
        self.client = OpenAI(api_key=api_key)
        self._model = model

    def converse(self, prompt: str, context: List[Dict], max_tokens: int = 250) -> str:
        context = self._remap(context)
        messages = context + [{"role": "user", "content": prompt}]
        response = self.client.chat.completions.create(
            model = self._model,
            messages = messages,
            max_tokens = max_tokens
        )
        return response.choices[0].message.content

    def _remap(self, context: List[Dict]) -> List[Dict]:
        ''' re-index the chat history to match the OpenAI API requirements '''
        new_context = []
        for statement in context:
            if statement["role"] == "_user":
                new_statement = {"role": "user", "content": statement["_content"]}
            elif statement["role"] == "_llm":
                new_statement = {"role": "assistant", "content": statement["_content"]}
            elif statement["role"] == "_preamble":
                new_statement = {"role": "system", "content": statement["_content"]}
            else:
                raise ValueError(f'Unknown chat role {statement["role"]}')
            new_context.append(new_statement)
        return new_context

We'll create 3 different session managers each with different levels of data access.
The narrowest scope is `session` where only prompts and responses passed to this manager are retrievable.
The next level is `user`, where all sessions of this specific user will be retrievable
The highesst level is `application` where all prompts and responses with the same application_id will be retrievable

In [14]:
from redisvl.extensions.session_manager import SessionManager
local_session = SessionManager(name='example', session_id='123', user_id='abc', application_id='travelbot_app', scope='session')
local_session.set_preamble("You are a helpful travel agent, assisting people in planning popular vacations")
client = OpenAIClient()

In [15]:
def converse(prompt:str, session_mngr: SessionManager):
    context = session_mngr.conversation_history()
    response = client.converse(prompt=prompt, context=context)
    session_mngr.store((prompt, response))
    print('USER: ', prompt)
    print('\nLLM: ', response)
    print('Conversation:')
    for ctx in context:
        print(ctx)

We can carry on several different conversations and inspect how data is accessible, or not accessible across different session instances.

In [16]:
converse("Can you help me plan a vacation to Scandinavia?", local_session)

USER:  Can you help me plan a vacation to Europe?

LLM:  Of course! Europe is a fantastic destination with a lot to offer. When are you thinking of traveling, and how long would you like to stay? Also, what are your interests and preferences? This information will help me suggest the best destinations and activities for your trip.
Conversation:
{'role': '_preamble', '_content': 'You are a helpful travel agent, assisting people in planning popular vacations'}


In [17]:
converse("I plan to travel this Fall for 2 weeks and really enjoy beautiful hikes. I have a reasonable, but not extravagant budget prefer active vacations.", local_session)

USER:  I plan to travel this Fall for 2 weeks and really enjoy beautiful hikes. I have a reasonable, but not extravagant budget prefer active vacations.

LLM:  Great! With your interest in beautiful hikes and active vacations, Europe has some stunning destinations to offer. Here are some suggestions for your 2-week trip this Fall:

1. **Switzerland**: Known for its picturesque landscapes and world-class hiking trails, Switzerland is a hiker's paradise. Consider visiting the Swiss Alps for challenging hikes with breathtaking views. The Jungfrau region and the Bernese Oberland offer some of the most stunning trails in the country.

2. **Scotland**: Explore the rugged beauty of the Scottish Highlands with its dramatic landscapes, ancient castles, and scenic trails. The West Highland Way is a popular long-distance hike that showcases the best of Scotland's natural beauty.

3. **Norway**: For a truly unique hiking experience, consider exploring the fjords and mountains of Norway. The Trollt

Unsurprisingly, the `local_session` mananger has access it it's own conversation history.
User level sessions with the same user id, in this case `abc` will also have access.
Let's create a new session with the same user, but different session id, and set its scope to `user`

In [18]:
user_session = SessionManager(name='example', session_id='456', user_id='abc', application_id='travelbot_app', scope='user')
user_session.set_preamble("You are a helpful travel agent, assisting people in planning popular vacations")
context = user_session.conversation_history()
for ctx in context:
    print(ctx)

16:18:15 redisvl.index.index INFO   Index already exists, not overwriting.
{'role': '_preamble', '_content': 'You are a helpful travel agent, assisting people in planning popular vacations'}
{'role': '_user', '_content': 'Can you help me plan a vacation to Europe?'}
{'role': '_llm', '_content': 'Of course! Europe is a fantastic destination with a lot to offer. When are you thinking of traveling, and how long would you like to stay? Also, what are your interests and preferences? This information will help me suggest the best destinations and activities for your trip.'}
{'role': '_user', '_content': 'I plan to travel this Fall for 2 weeks and really enjoy beautiful hikes. I have a reasonable, but not extravagant budget prefer active vacations.'}
{'role': '_llm', '_content': "Great! With your interest in beautiful hikes and active vacations, Europe has some stunning destinations to offer. Here are some suggestions for your 2-week trip this Fall:\n\n1. **Switzerland**: Known for its pictur

We can carry on the conversation here, or switch topics.

In [19]:
converse("that sounds great for week 1, what about for week 2?", user_session)

USER:  that sounds great for week 1, what about for week 2?

LLM:  For your second week in Europe, you might want to consider exploring some culturally rich cities that also offer opportunities for outdoor activities. Here are a few suggestions:

1. **Barcelona, Spain**: Barcelona is a vibrant city known for its unique architecture, delicious food, and lively atmosphere. You can take a break from hiking and explore the city's iconic landmarks such as the Sagrada Familia, Park Guell, and La Rambla. You can also relax on the city's beautiful beaches or go for a hike in the nearby mountains of Montserrat.

2. **Dubrovnik, Croatia**: Explore the stunning coastal city of Dubrovnik, known for its well-preserved medieval old town and crystal-clear waters. You can walk along the city walls for fantastic views of the Adriatic Sea, take a kayak tour along the coastline, or hike up Mount Srd for panoramic views of the city.

3. **Prague, Czech Republic**: Prague is a fairy-tale city with a rich h

The local session, with narrower scope, won't see this new conversation section.

In [20]:
for ctx in local_session.conversation_history():
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful travel agent, assisting people in planning popular vacations'}
{'role': '_user', '_content': 'Can you help me plan a vacation to Europe?'}
{'role': '_llm', '_content': 'Of course! Europe is a fantastic destination with a lot to offer. When are you thinking of traveling, and how long would you like to stay? Also, what are your interests and preferences? This information will help me suggest the best destinations and activities for your trip.'}
{'role': '_user', '_content': 'I plan to travel this Fall for 2 weeks and really enjoy beautiful hikes. I have a reasonable, but not extravagant budget prefer active vacations.'}
{'role': '_llm', '_content': "Great! With your interest in beautiful hikes and active vacations, Europe has some stunning destinations to offer. Here are some suggestions for your 2-week trip this Fall:\n\n1. **Switzerland**: Known for its picturesque landscapes and world-class hiking trails, Switzerland is a hiker's pa

Data is still compartmentalized to specific user_ids, so no information is available from different users either.

In [21]:
new_user_session = SessionManager(name='example', session_id='abc', user_id='def', application_id='travelbot_app')
converse("I'm planning a trip to southeast Asia. Which country in that region has the best hiking trails?", new_user_session)

16:18:21 redisvl.index.index INFO   Index already exists, not overwriting.
USER:  I'm planning a trip to southeast Asia. Which country in that region has the best hiking trails?

LLM:  Southeast Asia is home to some fantastic hiking trails, but one country that particularly stands out for its diverse and breathtaking landscapes is Nepal. Known for its majestic Himalayan mountain ranges, Nepal offers some of the most iconic trekking routes in the world, including the famous Annapurna Circuit and Everest Base Camp treks. 

Other countries in Southeast Asia with beautiful hiking trails include Vietnam, with its stunning mountainous regions like Sapa and Dalat, and Thailand, which offers trails through lush jungles and scenic national parks. Indonesia is also worth considering, especially the island of Bali with its volcanic landscapes and terraced rice fields. Each country in the region has its unique charm and appeal for hikers, so the best choice ultimately depends on your preferences a

There are some situations where one would want to see or share data across users.

In [22]:

app_session = SessionManager(name='example', session_id='789', user_id='def', application_id='travelbot_app', scope='application')
app_session.set_preamble("You are a data analist working for a travel agency, helping to track, monitor and report vacation trends.")
converse('based on you previous conversation history, what are the trending vacation places that people are asking you about?', app_session)



16:18:25 redisvl.index.index INFO   Index already exists, not overwriting.
USER:  based on you previous conversation history, what are the trending vacation places that people are asking you about?

LLM:  Based on our previous conversation history, the trending vacation destinations being asked about include:

1. Europe (Switzerland, Scotland, Norway, Austria, Italy, Spain, Croatia, Czech Republic)
    - Popular for its diverse landscapes, cultural heritage, and outdoor activities such as hiking and sightseeing.

2. Southeast Asia (Nepal, Vietnam, Thailand, Indonesia)
    - Known for its stunning natural beauty, diverse landscapes, and opportunities for adventurous activities like trekking and exploring.

These destinations offer a mix of outdoor adventures, cultural experiences, and scenic beauty, making them attractive options for travelers looking to immerse themselves in new and exciting vacation experiences.
Conversation:
{'role': '_preamble', '_content': 'You are a data analist w

In [23]:
app_session.conversation_history()

[{'role': '_preamble',
  '_content': 'You are a data analist working for a travel agency, helping to track, monitor and report vacation trends.'},
 {'role': '_user', '_content': 'Can you help me plan a vacation to Europe?'},
 {'role': '_llm',
  '_content': 'Of course! Europe is a fantastic destination with a lot to offer. When are you thinking of traveling, and how long would you like to stay? Also, what are your interests and preferences? This information will help me suggest the best destinations and activities for your trip.'},
 {'role': '_user',
  '_content': 'I plan to travel this Fall for 2 weeks and really enjoy beautiful hikes. I have a reasonable, but not extravagant budget prefer active vacations.'},
 {'role': '_llm',
  '_content': "Great! With your interest in beautiful hikes and active vacations, Europe has some stunning destinations to offer. Here are some suggestions for your 2-week trip this Fall:\n\n1. **Switzerland**: Known for its picturesque landscapes and world-clas

In [24]:
local_session.delete()